# Python et l'analyse de données spatiales
## livre de recettes pour exploiter des données géographiques avec Python
### version du 15 octobre 2018

### installation et chargement des librairies

La distribution Anaconda dispose de 150 librairies (packages) pour les data sciences déjà installés, et propose 250 packages additionnels
qui peuvent être facilement installés avec la commande 'conda install'

Nous utiliserons les packages suivants :
* Packages déjà installés avec la distribution Anaconda
    * pandas : structuration et manipulation de données sous forme de 'data frames'
    * matplotlib : graphiques
    * numpy : calcul scientifiques, tableaux multi dimensionnels
    * scikit-learn : analyses statistiques multivariées, apprentissage statistique
* Packages additionnels à installer :
    * psycopg2 : connexion à une base de données postgreSQL
    * geopandas : manipulation de données géographiques sous forme de data frames
    * osgeo : geotraitements avancés
    * folium : cartographie dynamique
        
Pour installer ces packages, exécuter dans le prompt d'Anaconda la commande `conda install nom_du_package`

**si cette commande ne marche pas**

C'est probablement un problème de proxy

Pour utiliser une installation avec conda, il faut d'abord rajouter dans le répertoire où est installé Anaconda un fichier .condarc permettant d'indiquer le proxy

Générer ce fichier avec le bloc notes, son contenu doit être :

    proxy_servers:
        http: http://direct.proxy.i2:8080
        https: https://direct.proxy.i2:8080

Une fois fait, lancer dans le prompt d'Anaconda la commande : `conda install nom_du_package`

### documentation et tutoriels sur ces librairies
* introduction à **pandas** : http://www.python-simple.com/python-pandas/panda-intro.php
* introduction à **matplotlib** : http://www.python-simple.com/python-matplotlib/graphes-multiples.php
* introduction à **numpy** : http://www.python-simple.com/python-numpy-scipy/creation-numpy.php
* géotraitements avec **gdal / ogr** (anglais) : https://www.gis.usu.edu/~chrisg/python/2009/
* cookbook **gdal / ogr** (anglais) : https://pcjericks.github.io/py-gdalogr-cookbook/index.html
* introduction à **scikit-learn** : https://makina-corpus.com/blog/metier/2017/initiation-au-machine-learning-avec-python-pratique
* documentation **geopandas** (anglais) : http://geopandas.org/
* documentation **folium** (anglais) : https://python-visualization.github.io/folium/index.html


In [ ]:
# chargement des packages
import psycopg2
import pandas as pd
import geopandas as gpd
import matplotlib as mp
import numpy as np
import matplotlib.pyplot as plt
import sklearn as skl
from osgeo import gdal
import folium
from folium.plugins import MarkerCluster

### connexion à une base de données PostgreSQL et lecture de tables

In [ ]:
# connexion à la base de données Ceremabase
dbcon = psycopg2.connect('host=172.20.52.153 port=5444 dbname=ceremabase user=consultation password=consultation')

In [ ]:
# lecture d'une table non géographique et copie dans un dataframe
data = pd.read_sql('SELECT * FROM w_gt.ressec4', con=dbcon) 

# cette table a un champ géographique mais avec Pandas il ne sera pas interprété comme une géométrie mais comme du texte

In [ ]:
# lecture d'une table géographique et copie dans un geodataframe
comgeo = gpd.read_postgis("SELECT * FROM r_admin_express.n_adm_exp_commune_000_2018 WHERE insee_dep LIKE '38'", con = dbcon, geom_col = 'geom')
datageo = gpd.read_postgis("SELECT * FROM e_ffonciers_2016.d38_2016_pb0010_local WHERE idcom LIKE '38193'", con=dbcon, geom_col = 'geomloc') 

# avec Geopandas, la colonne geom sera interprétée comme une géométrie

In [ ]:
# fermeture de la connexion
dbcon.close()

In [ ]:
# affichage des premières lignes d'un dataframe
data.head()

In [ ]:
# description statistique des variables du dataframe
# pour toutes les variables numériques du dataframe, renvoie nombre, moyenne, min, max, quartiles
datageo.describe()

### lecture d'un shapefile, lecture d'un raster

In [ ]:
# lecture d'un fichier shapefile
bati38 = gpd.GeoDataFrame.from_file('//CT69-SIGI/ref/BDTOPO/E_BATI/N_BATI_INDIFFERENCIE_BDT_038.SHP')

In [ ]:
# lecture d'un fichier raster
bdortho = gdal.Open('//CT69-SIGI/ref/BDORTHO/2015/D038-HR-ED15/38-2015-0895-6490-LA93-0M20-E080.jp2')

In [ ]:
# affichage des caractéristiques du raster
gt = bdortho.GetGeoTransform()
print(bdortho.GetMetadata()) # résolution du raster
print(bdortho.RasterXSize)   # taille X
print(bdortho.RasterYSize)   # taille Y
print(bdortho.RasterCount)   # nombre de bandes
print(gt)                    # coordonnées du point en haut à droite, résolution selon X et Y

In [ ]:
# affichage du raster

xmin, ymin, dx, dy = 0, 0, 200, 100 # définit le rectangle de pixels à afficher
datar = np.zeros((dy, dx, bdortho.RasterCount), dtype = 'uint8') # initialise un tableau numpy
for b in range(bdortho.RasterCount): # on boucle sur le nombre de bandes 
    bande = bdortho.GetRasterBand(b+1) # on extrait la bande b
    databande = bande.ReadAsArray(xmin, ymin, dx, dy) # copie la bande dans un tableau numpy à la taille du raster
    datar[:,:,b] = databande
plt.imshow(datar)

### création de graphiques avec Matplotlib

In [ ]:
# courbe
plt.plot(data.stoth[0:10]) # en se limitant au 10 premières lignes du dataframe (en Python l'indexation commence à zéro !)

In [ ]:
# nuages de points
plt.plot(data.stoth, data.spevtot, linestyle = 'none', marker ='o')
plt.xlabel('surface habitable')
plt.ylabel('surface totale')

In [ ]:
# graphique en barres (bar plot)
score = [29,11,20,10,330,213]
etiquettes = ['A', 'B', 'C', 'D', 'E', 'F']
plt.bar(range(len(score)), score) # ou plt.barh si on veut que les barres soient horizontales
plt.xticks([r for r in range(len(score))], etiquettes)

In [ ]:
# pour faire varier la largeur des barres proportionnellement à une 2e variable
largeur = [47,32,60,10,354,288] # largeur de chaque barre
espace = 10 # espacement entre les barres
position = [sum(largeur[0:i+1]) - largeur[i]/2 + i*espace for i in range(len(largeur))] # calcul de la position du centre de chaque barre
pourcentage = np.array(score) / np.array(largeur)
print(pourcentage)
plt.figure(figsize = (8,4))
plt.bar(position, score, width = largeur)
plt.xticks(position, etiquettes)

In [ ]:
# histogramme basique
plt.hist(data.jannath)

In [ ]:
# mise en forme de l'histogramme
plt.figure(figsize = (8,4)) # définit les dimensions du graphique
plt.hist(data.jannath[data.jannath > 1900], range = (1900, 2015), bins = 23, rwidth = 0.9, color = 'steelblue')
# on restreint la série de données à la condition jannath > 1900
# range : intervalle de l'axe des abscisses
# bins : nombre de classes dans cet intervalle
# rwidth : largeur des barres
# liste des couleurs disponibles sous matplotlib : http://www.python-simple.com/python-matplotlib/couleurs-matplotlib.php
plt.title('Titre du graphique')
plt.xlabel('Année')
plt.ylabel('Nombre')

In [ ]:
# camembert (pie plot)
d = data.groupby('locproptxt').size() # permet de compter le nombre de lignes pour chaque modalité du champ pivot
plt.figure(figsize = (12,4))
plt.axis('equal') # garantit que le camembert soit rond !
plt.pie(d, labeldistance = 0.5, autopct='%1.0f%%') # rajoute des étiquettes en pourcentage
plt.pie(d, labels = d.index, shadow = True) # dessine le camembert avec en étiquette les modalités
plt.legend(loc= 'upper left') # ajoute une légende

In [ ]:
# graphes multiples
fig = plt.figure(figsize=(12, 6))

ax = fig.add_subplot(1,2,1) # divise la figure horizontalement en 2 fenêtres (ax) et ouvre la fenetre 1
ax.hist(data.anneenaiss1[data.dqualp1=='MME'], range =(1900, 2000), bins = 20, rwidth = 0.9, orientation = 'horizontal', color ='pink')
plt.gca().invert_xaxis() # inversion de l'axe des x
plt.gca().invert_yaxis() # inversion de l'axe des y
ax.set_ylabel('annee de naissance')
ax.set_xlabel('nombre')
ax.set_title('femmes')

ax = fig.add_subplot(1,2,2) # ouvre la fenetre 2
ax.hist(data.anneenaiss1[data.dqualp1=='M'], range =(1900, 2000), bins = 20, rwidth = 0.9, orientation = 'horizontal', color ='steelblue')
plt.gca().invert_yaxis()
ax.set_ylabel('annee de naissance')
ax.set_xlabel('nombre')
ax.set_title('hommes')
ax.yaxis.set_label_position('right') # place le titre de l'axe y à droite
ax.yaxis.set_ticks_position('right') # place les graduations de l'axe y à droite

In [ ]:
# sauvegarde du graphique dans un fichier

plt.hist(data.jannath)
plt.savefig('image.png', dpi = 600, transparent = True)
# dpi fixe la résolution
# transparent = fond transparent
plt.close()

# on peut opter pour un autre format :    plt.savefig('image', format = 'pdf')

### création de cartes avec Geopandas

In [ ]:
# premières cartes basiques

comgeo.plot() # avec une géométrie polygon
datageo.plot(color='red', markersize=0.1) # ça marche pareil avec une couche de points

In [ ]:
# superposition de plusieurs couches
fig, ax = plt.subplots(figsize=(6, 6))
plt.axis('equal')
[xmin, ymin, xmax, ymax] = datageo.total_bounds # récupère les coordonnées limites de la couche datageo
marge = 1000
ax.set_xlim(xmin - marge, xmax + marge) # définit les limites de la carte en prenant une marge
ax.set_ylim(ymin - marge, ymax + marge)
comgeo.plot(ax=ax, color ='white', edgecolor ='black')
datageo.plot(ax=ax, color='red', markersize=0.1) # le plot se fait sur le même ax ce qui garantit la superposition des couches
comgeo.centroid.plot(ax=ax)


In [ ]:
# analyse thématique
fig = plt.figure(figsize=(16, 6))

ax = fig.add_subplot(1,2,1)
plt.axis('equal')
datageo[datageo.jannath > 1950].plot(ax=ax, column='jannath', legend = True, cmap = mp.cm.coolwarm, markersize=0.2)
# applique un style gradué selon la variable numérique jannath
# pour trouver la palette de couleurs adaptée, voir https://matplotlib.org/users/colormaps.html
ax.set_title("année de construction")
ax.xaxis.set_visible(False) # supprime l'affichage des étiquettes de l'axe x
ax.yaxis.set_visible(False) # supprime l'affichage des étiquettes de l'axe y

ax = fig.add_subplot(1,2,2)
plt.axis('equal')
datageo.plot(ax=ax, column='dteloctxt', legend = True, cmap = mp.cm.Set1, markersize=0.2)
# applique un style catégorisé selon la variable dteloctxt
ax.set_title("type de local")
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)


In [ ]:
# personnalisation de la barre de légende (colorbar)
fig = plt.figure(figsize=(16, 6))

# pour fixer manuellement les bornes min et max, utiliser les paramètres vmin et vmax
ax = fig.add_subplot(1,2,1)
datageo.plot(ax=ax, column='jannath', legend = True, vmin = 1960, vmax = 2010, cmap = mp.cm.coolwarm, markersize=0.2)

# pour personnaliser davantage la barre, il faut définir une colorbar manuellement
ax = fig.add_subplot(1,2,2)
vmin = 1960
vmax = 2010
datageo.plot(ax=ax, column='jannath', cmap = mp.cm.coolwarm, markersize=0.2, vmin = vmin, vmax = vmax) # par défaut legend = False

cax = fig.add_axes([0.5, 0.03, 0.4, 0.03]) # xmin, ymin, delta_x, delta_y
sm = plt.cm.ScalarMappable(cmap= mp.cm.coolwarm, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cb = fig.colorbar(sm, cax=cax, orientation = 'horizontal', ticks = [1960, 1980, 1990, 2000])


In [ ]:
# sélection par géométrie
fig, ax = plt.subplots(figsize=(6, 6))
comIdA = comgeo[comgeo.insee_com == '38193'] # on sélectionne la commune de l'Isle d'Abeau par slicing
comIdA.plot(ax=ax, color ='white', edgecolor ='black')
batiIdA = bati38.cx[870000:876000, 6502000:6506000] # permet de sélectionner les objets situés dans une "bounding box"
batiIdA = gpd.overlay(batiIdA, comIdA, how ='intersection') # équivalent à un SELECT ... WHERE ST_Intersects ...
batiIdA.plot(ax=ax, color = 'blue')

In [ ]:
# tampons
fig, ax = plt.subplots(figsize=(6, 6))
comIdA.plot(ax=ax, color ='white', edgecolor ='black')
tache = batiIdA.buffer(40) # tampon autour de chaque polygone
tache = tache.unary_union  # agrège en un seul multipolygone
tache = tache.buffer(-25)  # tampon négatif
gpd.plotting.plot_polygon_collection(ax=ax, geoms = tache) # on utilise la méthode plot_polygon_collection pour dessiner un multipolygone

### cartographie dynamique avec Folium

In [ ]:
# création d'un fond de carte
ma_carte = folium.Map(location=[45.6, 5.25], zoom_start = 10)

In [ ]:
# affichage de la carte
ma_carte

In [ ]:
# sauvegarde de la carte en html
ma_carte.save('ma_carte.html') # la carte est enregistrée dans le répertoire où se trouve ce notebook

In [ ]:
# pour rajouter une couche vectorielle, il faut d'abord reprojeter le geodataframe en long / lat
comgeo.crs = {'init' :'epsg:2154'} # définit la projection courante du geodataframe comgeo (lambert 93)
com4326 = comgeo.to_crs(epsg='4326') # reprojection en long / lat

In [ ]:
# ajout d'une couche vectorielle à la carte 
# (dans cet exemple ce sont des polygones, sans remplissage et avec une bordure orange)

folium.GeoJson(com4326, name='communes', style_function = lambda x: {'fillColor' : None, 'fillOpacity' : 0, 'color' : 'orange'}).add_to(ma_carte)

In [ ]:
# carte chloroplèthe à partir d'un geodataframe
# la méthode chloropleth() permet de réaliser une carte chloroplèthe dynamique, elle est plutôt conçue pour utiliser 
# des fichers json, mais fonctionne aussi si les données sont contenues dans un geodataframe

ma_carte.choropleth(
    geo_data=com4326,
    name='choroplèthe',
    data=com4326,
    columns=['id', 'population'], # le 1er champ est une clé, le 2e celui à partir duquel seront colorés les objets
    key_on='feature.properties.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    threshold_scale = [0,100,1000,5000,20000,1000000], # permet si on le souhaite d'indiquer les limites des classes
    legend_name='population par commune'
)

In [ ]:
# on peut aussi rajouter à la carte des objets ponctuels sous forme de markers
datageo.crs = {'init' :'epsg:2154'}
bati4326 = datageo[0:100].to_crs(epsg='4326') # on se limite volontairement à 100 bâtiments pour cet exemple


In [ ]:
# ajout de markers pour représenter des objects ponctuels

for row in bati4326.itertuples():
    folium.Marker(location = [row.geomloc.y, row.geomloc.x], popup = f"{row.dteloctxt} - {row.jannath} - {row.stoth} m2 hab.").add_to(ma_carte)

# une fenêtre popup s'ouvrira en cliquant sur le marker
# attention à ne pas vouloir afficher trop de markers sinon c'est très long (et illisible !)

** attention ** lorsque le texte du popup contient des caractères `' "` ou `& `

il faut utiliser une syntaxe du type `popup = folium.Popup("mon_texte", parse_html = True)`
sinon la carte ne s'affiche pas

In [ ]:
# groupage des markers (marker clusters)

mc = MarkerCluster()

for row in bati4326.itertuples():
    mc.add_child(folium.Marker(location = [row.geomloc.y, row.geomloc.x], popup = f"{row.dteloctxt} - {row.jannath} - {row.stoth} m2 hab."))
ma_carte.add_child(mc)

In [ ]:
# ajout à la carte d'un contrôle d'affichage des couches

folium.LayerControl().add_to(ma_carte)